In [ ]:
// general includes
#include <iostream>   // std::cout|endl 
#include <cstdio>   // std::printf 
#include <vector>   // std::vector
#include <list>   // std::list
#include <utility>   // std::pair
// using namespace std;
struct Widget {int m;};

## Quick recap: 

- iterators
  - iters annd classes
  - possibly lightwweight
  - interface:
    - move next/prev
    - get access to underlying value
    - comparison
- What is mssing
  - lambdas today, important
  - 

## Goals for today
- lambda: what is that

Lambda is:
- Nice language construct

# Lambda Expressions

A *lambda expression* [(cppref)](https://en.cppreference.com/w/cpp/language/lambda) constructs a *function object*, i.e., a class with an overloaded `operator()`, which is able to capture variables which are valid at the source location where the lambda expression is evaluated. 
This *function object* is of unnamed type and is ready to be invoked "like a function".
The required parameters for the invocation are specific within the lambda expression.
The invocation will execute the code block in the body of the lambda expression.
Let's look at a simple example where the full lambda expression and the variable for the function object (`lambda`) are marked:

In [ ]:
int i{};
Widget w{};  
auto lambda = [i, &w] (int b) -> int { return i + w.m + b; };
// int res = lambdas(4); // cling crash

**What is the effect of using `&w` compared to `w` in the capture clause?**
- capture "by-reference" instead of "by-copy"

**Can the variable names be altered during capturing?**

In [ ]:
int b = 5;
auto lambda1 = [b](int a) -> int { return a + b; }; // rename 'b' ?
auto lambda2 = [myb = b](int a) -> int { return a + myb; }; // copy and rename to 'myb'
auto lambda3 = [&myref = b](int a) -> int { return a + myref; }; // alias to 'b' named 'myref'

The effect of a lambda expression can nicely be described by the behavior using an inline class.
But before that, let's introduce some nomenclature for lambda expressions:

- The **body** holds the block of code which is executed when invoking the function object.
- The **capture clause** lists which variables of the "outside scope" are captured for a use inside the **body**.
- The **parameter list** lists the parameter which are required when invoking the function object; they are also available in the body (as for any normal function).
- The **return type** specifies the return type of the invocation.

The type of the **function object** which is constructed is of type "closure type" and its behavior is standardized by prescribing rules for
- how **captures** are mapped to members, 
- how the invocation is realized using an overloaded `operator()`, 
- which special member functions are available, and
- if conversion to a function pointer is possible (for non-capturing lambdas)

**Note**: the "closure type" is not directly accessible and compiler dependent; i.e. `auto` is used predominantly to deduce the type of function objects results from lambda expressions.

Considering the example from above and these rules just listed we could reformulate this

In [ ]:
int i{};
Widget w{};  
auto lambda = [i, &w] (int b) -> int { return i + w.m + b; };
// int res = lambdas(4); // cling crash

by using an "inline class" with 
- a custom constructor (1) which captures all required variables as members (1a)(1b),
- an overloaded `operator()` with the same *signature* as the "lambda" (2), and
- immediately instantiating the class (3):

In [ ]:
int i{};
Widget w{};
class ClosureType { 
private:
  int i;         // (1a) capture by-value
  Widget &w;     // (1b) capture by-reference
public:
  ClosureType(int i, Widget &w) : i(i), w(w) {};               // (1)
  auto operator()(int b) const -> int { return i + w.m + b;}   // (2)
};
ClosureType lambda(i,w); // (3)
int res = lambda(4);     // (4)

The effect and syntax of an invocation (4) is then identical to the above lambda expression.

**How does the inline class look like when the captures are empty? Does this allow for further simplification?**
- no data members -> stateless function object

**Can a lambda function be implicitly converted to a "C-Style" function pointer?**


In [ ]:
using FuncPtr = int (*)(int);
FuncPtr lambda5 = [](int a) -> int { return a + 2; }; // works? 
int b = 5;
// FuncPtr lambda6 = [b](int a) -> int { return a + b; }; // works?

Let's look at non-capturing lambdas in a bit more detail.

## Non-capturing lambdas

Non-capturing lambdas can implicitly be converted to function pointers; capturing lambdas cannot.
To create some intuition for the reason behind this behavior, let's first briefly look at function pointers and conversion functions.
Equipped with this prerequisites we then again mimic the behavior using an inline class which converts to a function pointer implicitly (like a lambda). 


### Function pointers

A *function pointer* is a type holding a pointer to a function with a specific *signature*.
Some examples of non-member function pointers are:

In [ ]:
using FuncPtrRetVoid = void (*)();
using FuncPtrRetIntPassIntInt= int(*)(int,int);
using FuncPtrRetWidgetPassWidgetRef = Widget (*)(const Widget&);

Invocation of the above function pointers looks like this:

In [ ]:
void func_void(){};

In [ ]:
int func_int(int a, int b) { return a + b; };

In [ ]:
Widget func_widget(const Widget &w) { return w; };

In [ ]:
FuncPtrRetVoid fptr = func_void;
fptr();
FuncPtrRetIntPassIntInt fptr2 = func_int;
fptr2(1, 2);
FuncPtrRetWidgetPassWidgetRef fptr3 = func_widget;
fptr3(Widget{});

A useful application of function pointers is to use them as parameters:

In [ ]:
void invoke(FuncPtrRetVoid callable) { callable(); }

invoke(func_void);
invoke(fptr);

### Conversion functions 

User-defined conversion functions [(cppref)](https://en.cppreference.com/w/cpp/language/cast_operator) allow to implement *implicit* and *explicit* conversions *form* a type *to* other types. 

**What is difference between explicit and implicit conversions?**
- a conversion is implicit if the type which is the target of the conversion is not explicitly stated in the code, but the required target type is deduced from the semantic embedding.
- an explicit conversion does prescribe the target type of the conversion independent of the semantic embedding.
- the specifier `explicit` can be used to disable the participation of a constructor or conversion function in implicit conversions.

A simple example of a conversion function looks like this:

In [ ]:
struct Widget{
  int m;
  operator int() { return m; }; // (1) conversion function from 'Widget' to 'int'
}; 
int d = Widget{3}; // implicit conversion used
int dd = 3 + Widget{3} * 4; // implicit conversion used

A second example where a conversion function (2) and a converting constructor [(cppref)](https://en.cppreference.com/w/cpp/language/converting_constructor) (1) participating in conversions looks like this:

In [ ]:
struct Widget1 {
  int m;
};

struct Widget2 {
  int m;
  Widget2() = default;
  // (1) converting constructor from 'Widget1' to 'Widget2'
  Widget2(const Widget1 &w) : m(w.m){};
  // (2) conversion function from 'Widget2' to 'Widget1'
  operator Widget1() { return Widget1{m}; };
};

Widget1 w1;
Widget2 w2;
w2 = w1; // implicit conversion using (1): error if (1) is specified 'explicit'
w2 = static_cast<Widget2>(w1);  // explicit conversion using (1)
w1 = w2; // implicit conversion function (2): error if (2) is specified 'explicit'
w1 = static_cast<Widget1>(w2);  // explicit conversion using (2)

### Lambda to function pointer conversion

For non-capturing lambdas, a conversion function can be used to allow a conversion to a function pointer. Let's look at a simple example again:

In [ ]:
using FuncPtrWW = Widget (*)(Widget&&);
FuncPtrWW fptr = [](Widget &&w) -> Widget {
  ++(w.m);
  return w;
};
// auto w = fptr(Widget{}); // cling crashes

The non-capturing lambda expression is implicitly converted to a function pointer. Also this conversion can be mimicked with an inline class:


In [ ]:
class ClosureType2 {
private:
  static auto invoke(Widget &&w) { // (1)
    ++(w.m);
    return w;
  }
public:
  operator FuncPtrWW() { return invoke; } // (2)
  auto operator()(Widget &&w) {
    ++(w.m);
    return w;
  }
};
FuncPtrWW fptr = ClosureType2();
auto w = fptr(Widget{});

A static member function (1) is additionally implemented, which is then returned  when a conversion to a function pointer is performed (2). 

**When is this conversion to a function pointer useful?**
- this conversion allows non-capturing lambdas to be used in situations requiring function pointers.
- this conversion provides a convenient way to define functions in a local scope.

**Why is this conversion not available if captures are present in the closure type?**
- captures effectively create a stateful local function object; invoking the lambda requires access to this object.
- a function does not have this access/object.

## Special member functions
Up to now we did not provide user-defined special member functions of the inline classes we used to mimic lambda expressions (beside a custom constructor). 
The standard prescribes rules for the SMFs for function objects constructed using lambdas: 

- default construction (1) is not available
- copy(2) and move(3) construction are declared as `default`ed
- copy assignment (4) is declared as `delete`d, (move assignment not declared)
- the destructor (6) is implicitly declared

Again mimicking these rule within the inline class this looks like this for the introductory example above:


In [ ]:
int i{};
Widget w{};
class ClosureType3 { 
private:
  int i;
  Widget &w;

public:

  ClosureType3() = delete; // (1) not defined
  ClosureType3(const ClosureType3 &) = default; // (2)
  ClosureType3(ClosureType3 &&) = default; // (3)
  ClosureType3 &operator=(const ClosureType3 &) = delete; // (4) 
  // ClosureType3 &operator=(ClosureType3&&); // (5) not declared
  ~ClosureType3() = default; // (6) implicit
  ClosureType3(int i, Widget &w) : i(i), w(w) {};   

  auto operator()(int b) const -> int { return i + w.m + b;}
};
ClosureType3 lambda(i,w);
// int res = lambda(4);

Exploring the effect of these rules for the SMFs can look like this:


In [ ]:
const int i{2};
Widget w{3};
auto lambda = [&i, &w](int b) { return i + w.m + b; };
// auto copy = lambda;              // what is this, does it work ? copy ctor
// lambda = copy;                // what is this, does it work ? copy assign
// auto move = std::move(lambda);   // what is this, does it work ? move ctor
// move = std::move(copy)           // what is this, does it work ? move assign  

## Evaluation time vs. execution time
For capturing lambdas, it is important to differentiate the time of *evaluation* of a lambda expression and the subsequent *invocations/executions* of the constructed function object:
the function object is constructed at *evaluation time* (1) of the lambda expression. At invocation time (2a)(2b) changes to variables captured by reference `&w` are reflected while the internal state of variables captured by-value (`x`) is effective. In (1) additionally the keyword `mutable` is required to allow the invocation to change the internal state (i.e., `++x`) of the function object.

In [ ]:
int x = 5; 
Widget w{7}; 

auto lambda = [x,&w]() mutable -> auto { ++x; return (w.m + x);}; // (1) 
x = 100; // change x to 100 in this scope
w.m = 100; // influences lambda -> lamnda.w.m == 100
// auto res1 = lambda(); // (2a) // cling crashes
x = 0; // reset in this scope (no effect on lambda)
w.m = 0; // effect on lambda 
// auto res2 = lambda(); // (2b) // cling crashes

**What are the values of `res1` and `res2` in above snippet?**
- `res1 = w.m + x;` where `x=5+1` and `w.m` = 7 
- `res2 = w.m + x;` where `x=6+1` and `w.m` = 0 

## Capture options
Above, up to now we only cherry-picked captures explicitly by-reference or by-value for local variables in the outer scope.

### Capture defaults/wildcards
To obtain a default capture by-value for "everything required" in the body of the lambda, a `=` as first item in the capture list can be used:


In [ ]:
double g;
const int i{2};
Widget w{3};
// default capture by-value    
auto lambda = [=](int b) { return g + w.m + b + i; }; 

Similar for a default-capture by lvalue reference, `&` as first item in the capture list can be used:

In [ ]:
// default capture by-reference    
auto lambda = [&](int b) { return g + w.m + b + i; }; 

The default can be mixed with explicit exceptions from the default:


In [ ]:
int i;
int g;

// explicit capture by-value for i,g
auto lambda1 = [&, i, g](int b) { return g + w.m + b + i; }; 
// explicit capture by-reference for i,g    
auto lambda2 = [=, &i, &g](int b) { return g + w.m + b + i; }; 

Further, names of variables can be provided explicitly:

In [ ]:
int i;
int g;
auto lambda = [=, i=i, ref_g=g](int b) { return i + ref_g + w.m + b + i; }; 

Moving-constructing an object into function object is achieved using this syntax:

In [ ]:
auto lambda = [=, moved_w=std::move(w)](int b) { return g + moved_w.m + b + i; }; 

### Enclosing object 

What happens if a lambda is defined in a scope with access to a `this` pointer and implicitly accesses the object pointed-to by `this`?
- if `this` is captured implicitly (through one of the two default capture modes `[=]` or `[&]`) the object referred to by this is captured by reference (see snippet below)



In [ ]:
struct Widget3 {
  int m;
  auto member() {
    return [&]() { return this->m; }; // (1) 'this' is captured by reference
    // return [=]() {return m; };       // (2) 'this' is still captured by reference
  }
};

To explicitly control type of capture for the enclosing object `this` can be captured by copy (1) or by reference (2):

In [ ]:
struct Widget4 {
  int m;
  auto member() {
    return [this]() { return m; }; // (1) by-reference 
    // return [*this]() { return m; }; // (2) by-copy
  }
};

## Generic lambdas

If one or more of the parameters use `auto` to specific the type the lambda expression is called *generic lambda*:


In [ ]:
const int i{2};
Widget4 w{3};
// generic lambda   
auto lambda1 = [=](auto b, double g) { return g + w.m + b + i; };
auto lambda2 = [=](auto & b, double g) { return g + w.m + b + i; }; 
auto lambda3 = [=](auto && b, double g) { return g + w.m + b + i; };

## Examples

Finally lets look at some examples.

## Custom comparator
This snippet uses lambdas to setup custom comparators which are the used to sort a vector of `Widgets`:

In [ ]:
struct Widget {
   int m;
};

std::vector<Widget> vec;
vec.push_back(Widget{10});
vec.push_back(Widget{5});
vec.push_back(Widget{4});
int threshold = 2;
// this comparator uses a local variable(s) to configure a threshold for the sorting
// this could not be easily achieved with a function pointer
auto comp1 = [threshold](const Widget &a, const Widget &b) {
  return std::abs(a.m - b.m) > threshold ? a.m < b.m : false;
};
auto comp2 = [](const Widget &a, const Widget &b) {
  return a.m < b.m;
};  
std::sort(vec.begin(), vec.end(), comp1); 
for (auto&& item : vec) {
  std::printf("%i ",item.m);
}

## Custom predicate

This snippet uses a lambda to setup a predicate which is used to count the `Widget`s in a vector, which have the property tested by the predicate:


In [ ]:
struct Widget {
  int m;
};
std::vector<Widget> vec;
vec.push_back(Widget{10});
vec.push_back(Widget{5});
vec.push_back(Widget{4});
int lower = 2;
int upper = 5;
// 
auto lambda = [&t1 = lower, &t2 = upper](const Widget &a) {
  return a.m > t1 && a.m < t2;
};
{ // count elements using predicate
  auto count = std::count_if(vec.begin(), vec.end(), lambda);
  std::printf("%li\n",count);
}
++upper; // increasing upper threshold of predicate
{ // count elements using predicate
  auto count = std::count_if(vec.begin(), vec.end(), lambda);
  std::printf("%li\n",count);
}

## Dangling capture
This snippet shows an example where a lambda captures a variable of the local scope by reference. This leads to a dangling reference if the lambda is invoked after this variable has gone out of scope:


In [ ]:
auto lambda = []() {
  int x{};
  return [&x]() { return x + 5; }; // capturing  local 'x' by reference
};
// auto res = lambda(); // cling crash
// lambda returns a dangling reference to the 'x' in the body of the lambda:
// reference was released after the operator() call completes


## Additional example

```cpp
// std::transform signature
template< class InputIt,
          class OutputIt,
          class UnaryOperation >
OutputIt transform( InputIt first1,
                    InputIt last1,
                    OutputIt d_first,
                    UnaryOperation unary_op );
```

Let's look at different example how to use the overloads (e.g., of `tan` in `cmath`) as unary operation when transforming a vector:

```cpp
// mimic overloads of 'tan '
double tan(double in) { return in; };
float tan(float in) { return in; };

// wrapper
template <typename T> auto tan_wrapper(const T &arg) { return tan(arg); }

int main() {
  using T = double;

  std::vector<T> v(10, 12.);

  // error: cannot infer template 'UnaryOperation'
  std::transform(v.begin(), v.end(), v.begin(), tan);

  // explicit template parameters
  std::transform<typename std::vector<T>::iterator,
                 typename std::vector<T>::iterator, double (*)(double)>(
      v.begin(), v.end(), v.begin(), tan);

  // explicit cast to function pointer
  std::transform(v.begin(), v.end(), v.begin(), static_cast<T (*)(T)>(tan));

  // using template wrapper
  std::transform(v.begin(), v.end(), v.begin(), tan_wrapper<T>);

  // using a lambda with auto argument-> templated operator() 
  std::transform(
      v.begin(), v.end(), v.begin(),
      [](const auto &a) -> auto { return tan(a); });
}
```


# Summary
- ...
- ...